In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%pip install igraph
import igraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.1 MB 15.4 MB/s 


# leiden communities

In [ ]:
G = igraph.Graph.Read_GML("/content/gdrive/MyDrive/reddit_graphs/twitter_filtered.gml")
#reddit = igraph.Graph.Read_GML("/content/gdrive/MyDrive/reddit_graphs/reddit_graph_v2.gml")

In [ ]:
v = G.community_leiden(weights = 'weight', resolution_parameter=1, objective_function='modularity')

In [ ]:
#adding node attribute indicating user's community
G.vs['community'] = v.membership

In [ ]:
# contract to smaller graph of communities
comm_G = v.cluster_graph(combine_edges=dict(weight="sum"))
comm_G.vcount() 

857

In [ ]:
# get size of each cluster
nodes_contained = [v.size(i) for i in range(len(v))]
comm_G.vs['size'] = nodes_contained

# w/ networkx

In [ ]:
import networkx as nx
# twitter_comms = igraph.Graph.to_networkx(comm_G)
# twitter = igraph.Graph.to_networkx(G)
twitter_comms = nx.read_gml("/content/gdrive/MyDrive/reddit_graphs/gml files/twitter_comms.gml")

In [ ]:
cc = sorted(nx.connected_components(twitter_comms), key=len, reverse=True)
twitter_comms = twitter_comms.subgraph(cc[0])
twitter_comms.number_of_nodes()

70

In [ ]:
#nx.write_gml(twitter_comms, "twitter_comms.gml")

In [ ]:
#figuring out which communities are kept
kept_comms = list(twitter_comms.nodes)

In [ ]:
#picking users in the kept communities
selected = [n for n,v in twitter.nodes(data=True) if v['community'] in kept_comms]
kept_twitter = twitter.subgraph(selected)

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(dict(kept_twitter.nodes(data=True)), orient='index')

,id,label,community
0,0.0,marcusp11,0
1,1.0,ctvottawa,13
2,2.0,Juliz1lb,0
3,3.0,QueenB_1967,0
4,4.0,LFPress,0


In [ ]:
df.to_csv("twitter_communities.csv", index = False)

# Merging sentiments with communities

In [ ]:
import pandas as pd
sentiments = pd.read_csv("/content/gdrive/MyDrive/reddit_graphs/twitter_communities_sentiments.csv")
sentiment_dict = dict(zip(sentiments['community'].astype(str), sentiments['sentiment']))
nx.set_node_attributes(twitter_comms, sentiment_dict, 'sentiment')

In [ ]:
nx.write_gml(twitter_comms, "twitter_comms.gml")